# Unknown classes

In [1]:
import numpy as np
import os
from pathlib import Path
import cv2
import pandas as pd
import tqdm
from imutils import build_montages

def extractLine(line, h, w):
    c, *xyxy = [float(x) for x in line.strip().split()]
    c = int(c)
    # print(line)
    # print(c)
    # print(xyxy)
    cx = xyxy[0]*w
    cy = xyxy[1]*h
    bw = xyxy[2]*w
    bh = xyxy[3]*h
    xmin = int(np.ceil(cx - bw/2))
    xmax = int(np.ceil(cx + bw/2))
    ymin = int(np.ceil(cy - bh/2))
    ymax = int(np.ceil(cy + bh/2))

    return c, xmin, xmax, ymin, ymax

def txt2df(datasetPath):
    df = pd.DataFrame(columns=['img', 'class', 'xmin', 'xmax', 'ymin', 'ymax'])
        
    labels_dir = Path(f'{datasetPath}/labels')

    for txt in tqdm.tqdm(os.listdir(labels_dir)):
        # img = cv2.imread(
        #             os.path.join(Path(f'{datasetPath}/images'),
        #             txt.replace('.txt','.jpg')
        #                         )
        #                     )
        # h, w = img.shape[:2]
        h, w = 480, 640
        # print(f'{h}x{w}')

        with open(Path(labels_dir).joinpath(txt)) as f:
            lines = f.readlines()

            for i, line in enumerate(lines):
                c, xmin, xmax, ymin, ymax = extractLine(line, h, w)

                df_row = pd.DataFrame(
                            {
                            'img': [txt],
                            'class': [c],
                            'xmin': [xmin],
                            'xmax': [xmax],
                            'ymin': [ymin],
                            'ymax': [ymax]
                            }
                        )

                df = pd.concat([df,df_row], ignore_index=True)

    csv_suffix = f'_{Path(datasetPath).name}'
    df.to_csv(f'df{csv_suffix}.csv', index=False)
    print('Done!')

In [25]:
def buildMontage(df, imgRoot, show=False, save=False, exp=None):
    images = []

    for imgPath in tqdm.tqdm(df.img):
        try:
            # print(imgPath)
            image = cv2.imread(os.path.join(imgRoot, imgPath.replace('txt', 'jpg')))
            # print(image)
            imgRow = df[df['img'] == imgPath]
            xmin = int(imgRow.xmin)
            ymin = int(imgRow.ymin)
            xmax = int(imgRow.xmax)
            ymax = int(imgRow.ymax)
            image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
            images.append(image)
        except:
            image = cv2.imread(os.path.join(imgRoot, imgPath.replace('txt', 'jpg')))
            # print(image)
            imgRow = df[df['img'] == imgPath]
            imgRows = imgRow.values.tolist()
            for imgRow in imgRows:
                # print(imgRow)
                xmin = int(imgRow[4])
                xmax = int(imgRow[5])
                ymin = int(imgRow[6])
                ymax = int(imgRow[7])
                image = cv2.rectangle(image, (xmin,ymin), (xmax,ymax), (0,255,0), 3)
            images.append(image)
            # pass

    montages = build_montages(images, (640//3, 640//4), (5,5))

    if show:
        showMontages(montages)

    if save:
        saveMontages(montages, exp)

    return montages

def showMontages(montages):
    for montage in montages:
        cv2.imshow('montage', montage)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

def saveMontages(montages, exp):
    if not exp:
        print("If you pass save as argument, please provide experiment name so the script can generate a path and save.")
    
    path = f'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\yolov5\\yolov5\\runs\\detect\\{exp}\\montages'

    if not os.path.isdir(path):
        os.mkdir(path)

    for idx, montage in enumerate(montages):
        montage_name = f'montage_{idx}.png'
        cv2.imwrite(os.path.join(path, montage_name), montage)

In [12]:
df_pred = pd.read_csv('bg_df_easy_uc_hybrid.csv')
df_pred.img.nunique()
df_pred

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.270462,0.271750,4,57,170,255
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.471748,0.473576,489,560,241,308
2,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.676251,0.678504,360,412,17,56
3,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.777861,0.778716,164,248,121,196
4,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.251937,0.254174,590,635,23,64
...,...,...,...,...,...,...,...,...
1084,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.713060,0.716859,429,503,104,143
1085,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.305016,0.307822,562,607,136,191
1086,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.342941,0.345203,445,516,104,147
1087,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.514308,0.517682,272,320,27,138


In [17]:
df_label = pd.read_csv('df_easy_uc_hybrid_gt.csv')
# df_label[df_label.cls_conf==1].img.nunique()
df_label

,img,cls,xmin,xmax,ymin,ymax,class
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,NaN,6,46,159,200,-3.0
1,07baf895b711927a078c12eb6deb88b0#201011270630_...,NaN,420,470,124,172,-3.0
2,07baf895b711927a078c12eb6deb88b0#201011270630_...,NaN,160,256,116,210,3.0
3,07baf895b711927a078c12eb6deb88b0#201011270630_...,NaN,362,414,14,59,3.0
4,07baf895b711927a078c12eb6deb88b0#201011270630_...,NaN,586,632,33,81,-3.0
...,...,...,...,...,...,...,...
2033,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,429,507,102,144,-4.0
2034,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,437,515,105,147,-4.0
2035,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,446,514,105,147,-4.0
2036,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,400,456,98,143,3.0


In [18]:
df_pred[df_pred.img.isin(df_label.img.unique())].shape[0] == df_pred.shape[0]

True

In [19]:
df_pred[df_pred.img.isin(df_label.img.unique())]

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.270462,0.271750,4,57,170,255
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.471748,0.473576,489,560,241,308
2,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.676251,0.678504,360,412,17,56
3,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.777861,0.778716,164,248,121,196
4,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.251937,0.254174,590,635,23,64
...,...,...,...,...,...,...,...,...
1084,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.713060,0.716859,429,503,104,143
1085,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.305016,0.307822,562,607,136,191
1086,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.342941,0.345203,445,516,104,147
1087,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.514308,0.517682,272,320,27,138


In [20]:
pathLabels = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_easy\\unknown_classes\\labels'
pathImages = pathLabels.replace('labels', 'images')
nameImages = sorted([f[:-len('.jpg')] for f in os.listdir(pathImages)])
nameLabelOG = sorted([f[:-len('.txt')] for f in os.listdir(pathLabels)])
namePred = sorted([f[:-len('.txt')] for f in df_pred.img.unique()])
nameLabels = sorted([f[:-len('.txt')] for f in df_label.img.unique()])

print(nameLabelOG==nameImages)
print(namePred==nameImages)
print(nameLabels==nameImages)
print(namePred==nameLabels)

namePredInImages = [f for f in namePred if f in nameImages]

print(len(namePred)==len(namePredInImages))

True
False
True
False
True


In [21]:
def box_iou(box1,box2):
    inter_w = ((min(box1[2], box2[2]) - max(box1[0], box2[0])))
    inter_h = ((min(box1[3], box2[3]) - max(box1[1], box2[1])))

    inter = inter_w*inter_h

    a1 = ((box1[2]-box1[0])*(box1[3]-box1[1]))
    a2 = ((box2[2]-box2[0])*(box2[3]-box2[1]))
    union = a1 + a2 - inter + 1e-12 #eps to avoid divide by 0 error

    return max(0, inter/union)

In [22]:
iou_th = 0.25
# print(tp_df.shape)
df_pred_iou = df_pred.copy()
df_pred_iou['iou'] = np.nan
df_pred_iou['label'] = np.nan
# Iterate through unique image IDs
for img in tqdm.tqdm(df_pred.img.unique()):
    df_pred_sub = df_pred[df_pred.img == img]
    df_label_sub = df_label[df_label.img == img].reset_index()

    # Calculate IoU for each pair of prediction and label bounding boxes in a vectorized manner
    pred_boxes = df_pred_sub[['xmin', 'ymin', 'xmax', 'ymax']].values
    label_boxes = df_label_sub[['xmin', 'ymin', 'xmax', 'ymax']].values

    for idx, pred_box in enumerate(pred_boxes):
        ious=[]
        for label_box in label_boxes:
            ious.append(box_iou(pred_box, label_box))
            # Update the 'iou' column with the maximum IoU for each prediction
        max_ious = np.max(ious)
        max_ious_index = ious.index(max_ious)
        # print(ious)
        # print(max_ious)
        # print(idx)
        # print(df_pred_sub[0])
        # print(df_pred_sub.index[idx])
        df_pred_iou.at[df_pred_sub.index[idx], 'iou'] = max_ious
        df_pred_iou.at[df_pred_sub.index[idx], 'label'] = df_label_sub['class'][max_ious_index]
        # print(df_label_sub['class'][max_ious_index])

# Filter out predictions based on the IoU threshold
df_pred_iou = df_pred_iou[df_pred_iou['iou'] >= iou_th]

# Remove the 'iou' column, if needed
# df_pred_iou = df_pred_iou.drop(columns=['iou'])

100%|██████████| 700/700 [00:01<00:00, 451.12it/s]


In [23]:
df_pred_iou

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,iou,label
2,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.676251,0.678504,360,412,17,56,17.102564,-3.0
3,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.777861,0.778716,164,248,121,196,4.496183,3.0
4,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.251937,0.254174,590,635,23,64,0.738739,-3.0
5,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.631617,0.632396,368,413,15,53,0.842105,3.0
6,07baf895b711927a078c12eb6deb88b0#201011270630_...,3,0.747835,0.749056,166,244,135,207,4.310345,6.0
...,...,...,...,...,...,...,...,...,...,...
1083,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.548761,0.552326,562,603,110,169,0.488889,-4.0
1084,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.713060,0.716859,429,503,104,143,0.880952,-4.0
1086,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.342941,0.345203,445,516,104,147,0.935473,-4.0
1087,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.514308,0.517682,272,320,27,138,0.795620,-4.0


In [26]:
imgPath = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_easy\\unknown_classes\\images'

buildMontage(df_pred_iou, imgPath, save=True, exp='easy_uc_hybrid')

100%|██████████| 1025/1025 [00:05<00:00, 180.44it/s]


[array([[[  4,   5,   9],
         [  7,  10,   0],
         [ 76,  87,  40],
         ...,
         [255, 255, 193],
         [186, 181, 148],
         [ 24,  14,   7]],
 
        [[  3,   2,   4],
         [  2,   3,   0],
         [ 99, 104,  77],
         ...,
         [255, 255, 192],
         [183, 179, 144],
         [ 24,  14,   7]],
 
        [[  7,   5,   5],
         [ 12,   9,   5],
         [198, 195, 187],
         ...,
         [255, 255, 190],
         [181, 178, 140],
         [ 25,  16,   7]],
 
        ...,
 
        [[  2,   1,   3],
         [  1,   4,   2],
         [  0,   1,   0],
         ...,
         [ 12,  22,  46],
         [  4,   7,  21],
         [  2,   0,   3]],
 
        [[  4,   4,   4],
         [  0,   4,   0],
         [  0,   6,   1],
         ...,
         [ 19,  31,  59],
         [  7,  12,  27],
         [  0,   0,   5]],
 
        [[  4,   4,   4],
         [  4,   8,   3],
         [  0,   3,   0],
         ...,
         [ 10,  25,  57],
  

In [27]:
# lowIouIdxs = np.arange(df_pred.shape[0]).tolist().remove(df_pred_iou.index.tolist())
lowIouIdxs = [item for item in df_pred.index if item not in df_pred_iou.index]
df_pred_lowIou = df_pred.iloc[lowIouIdxs]

In [14]:
buildMontage(df_pred.iloc[lowIouIdxs], imgPath, show=True)

100%|██████████| 253/253 [00:02<00:00, 109.94it/s]


[array([[[ 19,  21,   1],
         [ 19,  21,   0],
         [ 85,  89,  53],
         ...,
         [255, 237, 205],
         [103,  88,  79],
         [  7,   0,   5]],
 
        [[  5,   0,   0],
         [ 37,  31,  26],
         [164, 158, 151],
         ...,
         [255, 250, 218],
         [ 99,  87,  81],
         [ 14,   7,  14]],
 
        [[ 37,  26,  36],
         [ 41,  30,  40],
         [ 14,   0,  13],
         ...,
         [255, 253, 219],
         [101,  85,  78],
         [ 23,   9,  15]],
 
        ...,
 
        [[  0,   1,   0],
         [  0,   3,  10],
         [ 41,  55,  77],
         ...,
         [ 90, 104, 126],
         [ 21,  35,  41],
         [  0,   3,   0]],
 
        [[  3,   1,   0],
         [  0,   2,   7],
         [ 26,  40,  63],
         ...,
         [ 22,  35,  57],
         [  0,   1,   8],
         [  1,   8,   5]],
 
        [[ 23,  18,  19],
         [  0,   0,   7],
         [  2,  19,  40],
         ...,
         [ 69,  83, 105],
  

In [28]:
df_pred_iou.to_csv('easy_onUnknownSp_highIou.csv')
df_pred_lowIou.to_csv('easy_onUnknownSp_lowIou.csv')

# Test subset

In [29]:
df_pred = pd.read_csv('bg_df_easy_test_hybrid.csv')
df_pred

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.633493,0.638376,353,495,245,342
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.793037,0.798050,43,122,180,269
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.842882,0.850329,190,283,145,215
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.258479,0.260348,351,389,68,110
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.825986,0.827219,32,116,186,276
...,...,...,...,...,...,...,...,...
2412,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.550842,0.551963,180,236,169,220
2413,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.777157,0.777938,273,328,207,254
2414,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.337764,0.338614,235,290,226,263
2415,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.678152,0.679572,416,460,93,135


In [30]:
df_label = pd.read_csv('df_easy_test_hybrid_gt.csv')
# df_label[df_label.cls_conf==1].img.nunique()
df_label

,img,cls,xmin,xmax,ymin,ymax,class
0,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,NaN,339,395,17,54,3.0
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,NaN,334,378,62,104,3.0
2,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,NaN,190,286,149,217,0.0
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,NaN,39,121,182,269,0.0
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,NaN,175,281,135,210,0.0
...,...,...,...,...,...,...,...
2210,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,245,319,221,284,8.0
2211,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,274,326,209,254,3.0
2212,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,186,228,168,216,7.0
2213,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,NaN,416,460,93,135,3.0


In [34]:
df_pred[df_pred.img.isin(df_label.img.unique())].shape[0] == df_pred.shape[0]

False

In [32]:
df_pred[df_pred.img.isin(df_label.img.unique())].shape[0]

2246

In [36]:
df_pred.shape[0]

2417

In [37]:
pathLabels = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_easy\\test\\labels'
pathImages = pathLabels.replace('labels', 'images')
nameImages = sorted([f[:-len('.jpg')] for f in os.listdir(pathImages)])
nameLabelOG = sorted([f[:-len('.txt')] for f in os.listdir(pathLabels)])
namePred = sorted([f[:-len('.txt')] for f in df_pred.img.unique()])
nameLabels = sorted([f[:-len('.txt')] for f in df_label.img.unique()])

print(nameLabelOG==nameImages)
print(namePred==nameImages)
print(nameLabels==nameImages)
print(namePred==nameLabels)

namePredInImages = [f for f in namePred if f in nameImages]

print(len(namePred)==len(namePredInImages))

False
False
False
False
True


In [38]:
iou_th = 0.25
# print(tp_df.shape)
df_pred_iou = df_pred.copy()
df_pred_iou['iou'] = np.nan
df_pred_iou['label'] = np.nan

img_list = [img for img in df_pred.img.unique() if img in df_label.img.unique()]
# Iterate through unique image IDs
for img in tqdm.tqdm(img_list):
    df_pred_sub = df_pred[df_pred.img == img]
    df_label_sub = df_label[df_label.img == img].reset_index()

    # Calculate IoU for each pair of prediction and label bounding boxes in a vectorized manner
    pred_boxes = df_pred_sub[['xmin', 'ymin', 'xmax', 'ymax']].values
    label_boxes = df_label_sub[['xmin', 'ymin', 'xmax', 'ymax']].values

    for idx, pred_box in enumerate(pred_boxes):
        ious=[]
        for label_box in label_boxes:
            ious.append(box_iou(pred_box, label_box))
            # Update the 'iou' column with the maximum IoU for each prediction
        max_ious = np.max(ious)
        max_ious_index = ious.index(max_ious)
        # print(ious)
        # print(max_ious)
        # print(idx)
        # print(df_pred_sub[0])
        # print(df_pred_sub.index[idx])
        df_pred_iou.at[df_pred_sub.index[idx], 'iou'] = max_ious
        df_pred_iou.at[df_pred_sub.index[idx], 'label'] = df_label_sub['class'][max_ious_index]
        # print(df_label_sub['class'][max_ious_index])

# Filter out predictions based on the IoU threshold
df_pred_lowIou = df_pred_iou[df_pred_iou['iou'] < iou_th]
df_pred_iou = df_pred_iou[df_pred_iou['iou'] >= iou_th]

# Remove the 'iou' column, if needed
# df_pred_iou = df_pred_iou.drop(columns=['iou'])

100%|██████████| 1188/1188 [00:02<00:00, 466.83it/s]


In [39]:
df_pred_iou[df_pred_iou.label==df_pred_iou['class']]

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,iou,label
1,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.793037,0.798050,43,122,180,269,0.919637,0.0
4,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.825986,0.827219,32,116,186,276,0.922795,0.0
5,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.866304,0.868335,181,280,135,210,0.933962,0.0
7,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.818197,0.824317,37,116,188,271,0.897049,0.0
8,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.865027,0.867078,181,280,135,210,0.933962,0.0
...,...,...,...,...,...,...,...,...,...,...
2411,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,8,0.761523,0.766491,243,321,221,282,0.920098,8.0
2412,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.550842,0.551963,180,236,169,220,0.681159,7.0
2413,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.777157,0.777938,273,328,207,254,0.905222,3.0
2415,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.678152,0.679572,416,460,93,135,1.000000,3.0


In [43]:
df_pred_iou.to_csv('easy_test_highIou.csv')
df_pred_lowIou.to_csv('easy_test_lowIou.csv')

In [41]:
sorted(df_label['class'].unique())

[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0]

In [42]:
df_pred_lowIou

,img,class,cls_conf,obj_conf,xmin,xmax,ymin,ymax,iou,label
3,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.258479,0.260348,351,389,68,110,0.224474,0.0
6,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.292806,0.294835,351,390,68,110,0.223271,0.0
9,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.262964,0.264413,357,392,74,114,0.178581,0.0
80,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,0,0.287925,0.289536,7,63,129,192,0.217361,3.0
195,01465f8f61db58564cd37ce2dfc519c5#201106090830_...,3,0.291801,0.292992,485,543,181,221,0.093721,3.0
...,...,...,...,...,...,...,...,...,...,...
2332,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.511994,0.514383,387,434,34,74,0.000000,3.0
2335,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.251880,0.253064,339,387,27,63,0.197861,3.0
2340,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,3,0.279095,0.279919,99,142,44,83,0.000000,3.0
2372,fe16e25855b7fe76e58dd968b328e09b#201104140610_...,7,0.276047,0.290326,57,115,104,141,0.123818,3.0


In [47]:
imgPath = 'C:\\Users\\leofi\\OneDrive - Universidade de Lisboa\\Documents\\GitHub\\masters\\Data\\fc2015_yolo_uc_easy\\test\\images'
buildMontage(df_pred_lowIou, imgPath, show=True)

100%|██████████| 58/58 [00:00<00:00, 112.73it/s]


[array([[[  5,  10,  19],
         [ 32,   7,  11],
         [123,  74,  72],
         ...,
         [255, 255, 198],
         [199, 193, 152],
         [ 33,  15,   8]],
 
        [[  0,   0,   7],
         [ 33,  12,  15],
         [174, 137, 133],
         ...,
         [255, 255, 196],
         [198, 192, 149],
         [ 32,  14,   7]],
 
        [[ 44,  40,  51],
         [ 68,  45,  49],
         [ 50,  12,   8],
         ...,
         [255, 255, 195],
         [197, 191, 148],
         [ 32,  15,   6]],
 
        ...,
 
        [[  0,   1,   2],
         [ 14,   6,  17],
         [ 27,  17,  30],
         ...,
         [ 84, 119,  55],
         [ 31,  53,  25],
         [  0,   5,   8]],
 
        [[  0,   1,   2],
         [ 14,   6,  17],
         [ 27,  16,  32],
         ...,
         [ 37,  63,  27],
         [  0,  13,   7],
         [  0,   1,  20]],
 
        [[  0,   1,   2],
         [ 14,   6,  17],
         [ 28,  17,  33],
         ...,
         [ 37,  56,  39],
  